In [ ]:
from snowflake.snowpark.context import get_active_session
from snowflake.snowpark.functions import col, dateadd, current_date, lit
import pandas as pd, numpy as np

session = get_active_session()

df = session.table("WELLS.MINERALS.DISTINCT_OWNERS_WITH_DATA").to_pandas()
wells = list({val for row in df['API_LIST'] for val in str(row).split(',')})

In [ ]:
def get_prd_data(selected_well):   
    sql = f"""
        SELECT *
        FROM RAW_PROD_DATA
        WHERE API_UWI = '{selected_well}'
    """
    result = session.sql(sql).collect()
    df = pd.DataFrame([row.asDict() for row in result])
    df['PRODUCINGMONTH'] = pd.to_datetime(df['PRODUCINGMONTH'])
    # 2. Sort by date
    df = df.sort_values('PRODUCINGMONTH').reset_index(drop=True)
    # 3. Add month index (0 to n)
    df['Month'] = (df['PRODUCINGMONTH'] - df['PRODUCINGMONTH'].min()).dt.days // 30
    return df

In [ ]:
def get_econ_parameters(selected_well):
    sql = f"""
        SELECT *
        FROM ECON_INPUT_1PASS
        WHERE API_UWI = '{selected_well}'
    """
    params = session.sql(sql).to_pandas()
    params = params.sort_values('LAST_EDIT_DATE')
    params = params.iloc[0].to_dict()
    return params

In [ ]:
def calculate_decline_rates(qi, qf, decline_type, b_factor=None, initial_decline=None, terminal_decline=None, max_months=600):
    """
    Calculate production rates using either exponential or hyperbolic decline.
    For hyperbolic decline, switches to exponential at terminal decline rate.
    Returns tuple of (rates array, decline_types array)
    """
    rates = []
    decline_types = []
    current_rate = qi
    # 189.5 nan EXP nan 0.32155217278441517 0.04
    if decline_type == 'EXP':  # Pure exponential decline
        monthly_decline = 1 - np.exp(-initial_decline/12)
        
        while current_rate > qf and len(rates) < max_months:
            rates.append(current_rate)
            current_rate *= (1 - monthly_decline)
            
    else:  # Hyperbolic decline with terminal transition
        t = 0
        monthly_terminal = 1 - np.exp(-terminal_decline/12)
        
        while current_rate > qf and len(rates) < max_months:
            # Calculate current annual decline rate
            current_decline = initial_decline / (1 + b_factor * initial_decline * t/12)
            
            # Check for transition to terminal decline
            if current_decline <= terminal_decline:
                # Switch to exponential decline using terminal decline rate
                while current_rate > qf and len(rates) < max_months:
                    rates.append(current_rate)
                    current_rate *= (1 - monthly_terminal)
                break
            
            # Still in hyperbolic decline
            rates.append(current_rate)
            # Calculate next rate using hyperbolic formula
            current_rate = qi / np.power(1 + b_factor * initial_decline * (t + 1)/12, 1/b_factor)
            t += 1
    
    return np.array(rates)

In [ ]:
def calc_decline(prd, params):
    # Get the last production date
    last_prod_date = prd['PRODUCINGMONTH'].max()
    # Get forecast start dates from ECON_INPUT
    oil_fcst_start = params['FCST_START_OIL']
    gas_fcst_start = params['FCST_START_GAS']
    # Ensure fost_starecast start dates are on the first day of the month
    oil_fcst_start = oil_fcst_start.replace(day=1)
    gas_fcst_start = gas_fcst_start.replace(day=1)
    earliest_fcst_start = min(oil_fcst_start, gas_fcst_start)
    gas_qi = 0 if pd.isna(params.get('GAS_CALC_QI')) else params.get('GAS_CALC_QI')
    gas_qf = 0 if pd.isna(params.get('GAS_Q_MIN')) else params.get('GAS_Q_MIN')
    gas_decline_type = "EXP" if pd.isna(params.get('GAS_DECLINE_TYPE')) else params.get('GAS_DECLINE_TYPE')
    gas_decline = 0 if pd.isna(params.get('GAS_EMPIRICAL_DI')) else params.get('GAS_EMPIRICAL_DI')
    gas_b_factor = 0.8 if pd.isna(params.get('GAS_CALC_B_FACTOR')) or params.get('GAS_CALC_B_FACTOR') < 0.8 else params.get('GAS_CALC_B_FACTOR')
    terminal_decline = 0 if pd.isna(params.get('GAS_D_MIN')) else params.get('GAS_D_MIN')

    gas_rates = calculate_decline_rates(gas_qi, 
                                        gas_qf, 
                                        gas_decline_type, 
                                        gas_b_factor, 
                                        gas_decline, 
                                        terminal_decline, 
                                        max_months=600
                                        )
    # Calculate the index offset for gas forecast start
    gas_offset = (gas_fcst_start - earliest_fcst_start).days // 30
    gas_offset = max(0, gas_offset)  # Ensure non-negative
    max_months = 600
    dates = pd.date_range(start=earliest_fcst_start, periods=max_months, freq='MS')
    well_fcst = pd.DataFrame({'PRODUCINGMONTH': dates})
    # Limit to available array length
    data_length = min(len(gas_rates), len(well_fcst) - gas_offset)
    # Insert gas forecast at the correct position
    well_fcst.loc[gas_offset:gas_offset+data_length-1, 'GasFcst_MCF'] = gas_rates[:data_length]
    # Add gas forecast start date for reference
    well_fcst['GasFcst_Start_Date'] = gas_fcst_start


    oil_qi = 0 if pd.isna(params.get('OIL_CALC_QI')) else params.get('OIL_CALC_QI')
    oil_qf = 0 if pd.isna(params.get('OIL_Q_MIN')) else params.get('OIL_Q_MIN')
    oil_decline_type = params.get('OIL_DECLINE_TYPE')
    oil_decline = 0 if pd.isna(params.get('OIL_EMPIRICAL_DI')) else params.get('OIL_EMPIRICAL_DI')
    oil_b_factor = 0.8 if pd.isna(params.get('OIL_CALC_B_FACTOR'))  or params.get('OIL_CALC_B_FACTOR') < 0.8 else params.get('OIL_CALC_B_FACTOR')
    oil_terminal_decline = 0 if pd.isna(params.get('OIL_D_MIN')) else params.get('OIL_D_MIN')

    oil_rates = calculate_decline_rates(
        qi=oil_qi,
        qf=oil_qf,
        decline_type=oil_decline_type,
        b_factor=oil_b_factor,
        initial_decline=oil_decline,
        terminal_decline=oil_terminal_decline, 
        max_months=600
    )

    # Calculate the index offset for oil forecast start
    oil_offset = (oil_fcst_start - earliest_fcst_start).days // 30
    oil_offset = max(0, oil_offset)  # Ensure non-negative
    data_length = min(len(oil_rates), len(well_fcst) - oil_offset)
    well_fcst.loc[oil_offset:oil_offset+data_length-1, 'OilFcst_BBL'] = oil_rates[:data_length]
    # Add oil forecast start date for reference
    well_fcst['OilFcst_Start_Date'] = oil_fcst_start

    # Initialize all possible forecast columns to ensure they exist
    forecast_cols = ['GasFcst_MCF', 'OilFcst_BBL']
    for col in forecast_cols:
        if col not in well_fcst.columns:
            well_fcst[col] = np.nan
            
    final_df = pd.merge(prd, well_fcst, on='PRODUCINGMONTH', how='outer')
    # Filter to end of 2050
    final_df = final_df[final_df['PRODUCINGMONTH'] <= '2050-12-31']
    final_df = final_df.sort_values('PRODUCINGMONTH')
    gas_cutoff_date = final_df['PRODUCINGMONTH'].min() + pd.DateOffset(years=params['GAS_FCST_YRS'])
    final_df.loc[final_df['PRODUCINGMONTH'] > gas_cutoff_date, 'GasFcst_MCF'] = None
    final_df['API_UWI'] = final_df['API_UWI'].fillna(final_df['API_UWI'].unique()[0])
    return final_df

In [ ]:
import pandas as pd
import numpy as np
from datetime import date as _date

def calc_decline(prd, params):

    last_prod_date = prd['PRODUCINGMONTH'].max()

    oil_fcst_start = params['FCST_START_OIL']
    gas_fcst_start = params['FCST_START_GAS']

    oil_fcst_start = oil_fcst_start.replace(day=1) if pd.notna(oil_fcst_start) else None
    gas_fcst_start = gas_fcst_start.replace(day=1) if pd.notna(gas_fcst_start) else None

    # Base forecast frame
    max_months = 600
    earliest_fcst_start = min(
        d for d in [oil_fcst_start, gas_fcst_start] if d is not None
    ) if any([oil_fcst_start, gas_fcst_start]) else None

    if earliest_fcst_start is None:
        # No forecast dates → all zero
        well_fcst = prd[['PRODUCINGMONTH']].drop_duplicates().copy()
        well_fcst['GasFcst_MCF'] = 0
        well_fcst['OilFcst_BBL'] = 0
        well_fcst['GasFcst_Start_Date'] = None
        well_fcst['OilFcst_Start_Date'] = None
    else:
        dates = pd.date_range(start=earliest_fcst_start, periods=max_months, freq='MS')
        well_fcst = pd.DataFrame({'PRODUCINGMONTH': dates})

        # --- GAS ---
        if gas_fcst_start is None:
            well_fcst['GasFcst_MCF'] = 0
            well_fcst['GasFcst_Start_Date'] = None
        else:
            gas_qi = 0 if pd.isna(params.get('GAS_CALC_QI')) else params['GAS_CALC_QI']
            gas_qf = 0 if pd.isna(params.get('GAS_Q_MIN')) else params['GAS_Q_MIN']
            gas_decline_type = "EXP" if pd.isna(params.get('GAS_DECLINE_TYPE')) else params['GAS_DECLINE_TYPE']
            gas_decline = 0 if pd.isna(params.get('GAS_EMPIRICAL_DI')) else params['GAS_EMPIRICAL_DI']
            gas_b_factor = 0.8 if pd.isna(params.get('GAS_CALC_B_FACTOR')) or params['GAS_CALC_B_FACTOR'] < 0.8 else params['GAS_CALC_B_FACTOR']
            terminal_decline = 0 if pd.isna(params.get('GAS_D_MIN')) else params['GAS_D_MIN']

            gas_rates = calculate_decline_rates(
                gas_qi, gas_qf, gas_decline_type, gas_b_factor, gas_decline, terminal_decline, max_months=max_months
            )
            gas_offset = max(0, (gas_fcst_start - earliest_fcst_start).days // 30)
            data_length = min(len(gas_rates), len(well_fcst) - gas_offset)
            well_fcst['GasFcst_MCF'] = np.nan
            well_fcst.loc[gas_offset:gas_offset+data_length-1, 'GasFcst_MCF'] = gas_rates[:data_length]
            well_fcst['GasFcst_Start_Date'] = gas_fcst_start

        # --- OIL ---
        if oil_fcst_start is None:
            well_fcst['OilFcst_BBL'] = 0
            well_fcst['OilFcst_Start_Date'] = None
        else:
            oil_qi = 0 if pd.isna(params.get('OIL_CALC_QI')) else params['OIL_CALC_QI']
            oil_qf = 0 if pd.isna(params.get('OIL_Q_MIN')) else params['OIL_Q_MIN']
            oil_decline_type = params.get('OIL_DECLINE_TYPE')
            oil_decline = 0 if pd.isna(params.get('OIL_EMPIRICAL_DI')) else params['OIL_EMPIRICAL_DI']
            oil_b_factor = 0.8 if pd.isna(params.get('OIL_CALC_B_FACTOR')) or params['OIL_CALC_B_FACTOR'] < 0.8 else params['OIL_CALC_B_FACTOR']
            oil_terminal_decline = 0 if pd.isna(params.get('OIL_D_MIN')) else params['OIL_D_MIN']

            oil_rates = calculate_decline_rates(
                oil_qi, oil_qf, oil_decline_type, oil_b_factor, oil_decline, oil_terminal_decline, max_months=max_months
            )
            oil_offset = max(0, (oil_fcst_start - earliest_fcst_start).days // 30)
            data_length = min(len(oil_rates), len(well_fcst) - oil_offset)
            well_fcst['OilFcst_BBL'] = np.nan
            well_fcst.loc[oil_offset:oil_offset+data_length-1, 'OilFcst_BBL'] = oil_rates[:data_length]
            well_fcst['OilFcst_Start_Date'] = oil_fcst_start

    # Merge with prod data
    final_df = pd.merge(prd, well_fcst, on='PRODUCINGMONTH', how='outer')
    final_df = final_df[final_df['PRODUCINGMONTH'] <= '2050-12-31'].sort_values('PRODUCINGMONTH')

    # Gas cutoff
    if 'GAS_FCST_YRS' in params and pd.notna(params['GAS_FCST_YRS']):
        gas_cutoff_date = final_df['PRODUCINGMONTH'].min() + pd.DateOffset(years=params['GAS_FCST_YRS'])
        final_df.loc[final_df['PRODUCINGMONTH'] > gas_cutoff_date, 'GasFcst_MCF'] = None

    final_df['API_UWI'] = final_df['API_UWI'].fillna(final_df['API_UWI'].unique()[0])

    # Clean date columns
    for _c in ['GasFcst_Start_Date', 'OilFcst_Start_Date']:
        final_df[_c] = pd.to_datetime(final_df[_c], errors='coerce').dt.date
        final_df[_c] = final_df[_c].fillna(_date(1900, 1, 1))

    # <<< Add ECON_SCENARIO from params >>>
    scenario = params.get('ECON_SCENARIO') if hasattr(params, 'get') else params['ECON_SCENARIO']
    final_df['ECON_SCENARIO'] = scenario

    return final_df


In [ ]:
# i = 0
# for well in wells:
#     i+=1
#     print(f"{i} - {well}")
#     # if well == '4216131968':
#     api = f"{well[0:2]}-{well[2:5]}-{well[5:]}"
#     prd = get_prd_data(api)
#     params = get_econ_parameters(api)
#     forecast_df = calc_decline(prd, params)
#     forecast_df = forecast_df.reset_index(drop=True).copy()
#     forecast_df['PRODUCINGMONTH'] = pd.to_datetime(forecast_df['PRODUCINGMONTH']).dt.date  # safer type
#     # write (robust unpack)
#     res = session.write_pandas(forecast_df, "FORECASTS", auto_create_table=True)
#     success, nchunks, nrows, *rest = res 
#     # break
# # forecast_df

In [ ]:
# ## calcu forecast for specific wells
# from snowflake.snowpark.context import get_active_session
# import pandas as pd

# session = get_active_session()

# # Wells that returned “No production data found”
# missing_apis = ['42-041-32602', '42-041-32540', '42-041-32515']

# for api in missing_apis:
#     try:
#         prd = get_prd_data(api)
#         if prd.empty:
#             print(f"{api}: no rows in RAW_PROD_DATA")
#             continue

#         params = get_econ_parameters(api)
#         if not params:
#             print(f"{api}: no rows in ECON_INPUT_1PASS")
#             continue

#         fcst = calc_decline(prd, params).reset_index(drop=True)
#         fcst["PRODUCINGMONTH"] = pd.to_datetime(fcst["PRODUCINGMONTH"]).dt.date
#         fcst["API_UWI"] = api  # ensure well identifier is present

#         # Append forecast rows to the FORECASTS table
#         session.write_pandas(fcst, "FORECASTS", auto_create_table=True, overwrite=False)
#         print(f"{api}: wrote {len(fcst)} forecast rows")

#     except Exception as exc:
#         print(f"{api}: failed → {exc}")


In [ ]:
from snowflake.snowpark.context import get_active_session
from snowflake.snowpark.functions import col
import pandas as pd

session = get_active_session()
owner = "RELYEA TIM & MIRIAM"

# Lookup API numbers (no dashes) for this owner
df = (
    session.table("WELLS.MINERALS.DISTINCT_OWNERS_WITH_DATA")
           .filter(col('"Owner"') == owner)
           .to_pandas()
)
wells = {
    api for apis in df["API_LIST"] for api in str(apis).split(",") if api.strip()
}

# Normalize to dashed API_UWI format
apis = [f"{raw_api[0:2]}-{raw_api[2:5]}-{raw_api[5:]}" for raw_api in wells]

# Delete any existing forecasts for these wells
if apis:
    in_clause = ",".join(f"'{api}'" for api in apis)
    session.sql(f"DELETE FROM FORECASTS WHERE API_UWI IN ({in_clause})").collect()
    print(f"Deleted old forecast rows for {len(apis)} wells")

# Recalculate and upload forecasts
for api in apis:
    try:
        prd = get_prd_data(api)
        if prd.empty:
            print(f"{api}: no production data → skipped")
            continue

        params = get_econ_parameters(api)
        if not params:
            print(f"{api}: no econ params → skipped")
            continue

        fcst = calc_decline(prd, params).reset_index(drop=True)
        fcst["PRODUCINGMONTH"] = pd.to_datetime(fcst["PRODUCINGMONTH"]).dt.date
        fcst["API_UWI"] = api

        session.write_pandas(fcst, "FORECASTS", auto_create_table=True, overwrite=False)
        print(f"{api}: wrote {len(fcst)} forecast rows")

    except Exception as exc:
        print(f"{api}: failed → {exc}")


In [ ]:
-- CREATE OR REPLACE TABLE FORECASTS (
--   "WELLID" NUMBER(38,0),
--   "COMPLETIONID" NUMBER(38,0),
--   "API_UWI" VARCHAR,
--   "UNFORMATTED_API_UWI" NUMBER(38,0),
--   "WELLNAME" VARCHAR,
--   "WELLBOREID" NUMBER(38,0),
--   "PRODUCINGMONTH" TIMESTAMP_NTZ,
--   "TOTALPRODMONTHS" NUMBER(38,0),
--   "TOTALCOMPLETIONMONTHS" NUMBER(38,0),
--   "PRODUCINGDAYS" NUMBER(38,0),
--   "PROD_BOE" NUMBER(38,0),
--   "PROD_MCFE" NUMBER(38,0),
--   "LIQUIDSPROD_BBL" NUMBER(38,0),
--   "GASPROD_MCF" NUMBER(38,0),
--   "WATERPROD_BBL" NUMBER(38,0),
--   "CONDENSPROD_BBL" NUMBER(38,0),
--   "NGLPROD_BBL" NUMBER(38,0),
--   "CO2PROD_MCF" NUMBER(38,0),
--   "H2SPROD_MCF" NUMBER(38,0),
--   "SSPROD_MCF" NUMBER(38,0),
--   "HELIUMPROD_MCF" NUMBER(38,0),
--   "CALENDARDAYLIQUIDS_BBLPERDAY" NUMBER(38,0),
--   "CALENDARDAYGAS_MCFPERDAY" NUMBER(38,0),
--   "CALENDARDAYWATER_BBLPERDAY" NUMBER(38,0),
--   "CALENDARDAYCONDENSATE_BBLPERDAY" NUMBER(38,0),
--   "CALENDARDAYNGL_BBLPERDAY" NUMBER(38,0),
--   "CALENDARDAYBOE_BBLPERDAY" NUMBER(38,0),
--   "CALENDARDAYMCFE_MCFPERDAY" NUMBER(38,0),
--   "CALENDARDAYCO2_MCFPERDAY" NUMBER(38,0),
--   "CALENDARDAYH2S_MCFPERDAY" NUMBER(38,0),
--   "CALENDARDAYSS_MCFPERDAY" NUMBER(38,0),
--   "CALENDARDAYHELIUM_MCFPERDAY" NUMBER(38,0),
--   "GASINJECTION_MCF" NUMBER(38,0),
--   "WATERINJECTION_BBL" NUMBER(38,0),
--   "STEAMINJECTION_BBL" NUMBER(38,0),
--   "STEAMINJECTION_TON" NUMBER(38,0),
--   "SOLVENTINJECTION_BBL" NUMBER(38,0),
--   "OTHERINJECTION_BBL" NUMBER(38,0),
--   "GASLIFT_MCF" NUMBER(38,0),
--   "CASINGPRESSURE_PSI" NUMBER(38,0),
--   "TUBINGPRESSURE_PSI" NUMBER(38,0),
--   "FLOWLINEPRESSURE_PSI" NUMBER(38,0),
--   "WELLHEADPRESSURE_PSI" NUMBER(38,0),
--   "GASGRAVITY" NUMBER(38,0),
--   "OILGRAVITY" NUMBER(38,0),
--   "INJECTIONGAS_MCF" NUMBER(38,0),
--   "INJECTIONSOLVENT_BBL" NUMBER(38,0),
--   "INJECTIONSTEAM_BBL" NUMBER(38,0),
--   "INJECTIONWATER_BBL" NUMBER(38,0),
--   "INJECTIONOTHER_BBL" NUMBER(38,0),
--   "CALENDARDAYINJECTIONWATER_BBLPERDAY" NUMBER(38,0),
--   "CALENDARDAYINJECTIONSTEAM_BBLPERDAY" NUMBER(38,0),
--   "CALENDARDAYINJECTIONSOLVENT_BBLPERDAY" NUMBER(38,0),
--   "CALENDARDAYINJECTIONGAS_MCFPERDAY" NUMBER(38,0),
--   "CALENDARDAYINJECTIONOTHER_BBLPERDAY" NUMBER(38,0),
--   "ENVPRODID" NUMBER(38,0),
--   "Month" NUMBER(38,0),
--   "GasFcst_MCF" FLOAT,
--   "GasFcst_Start_Date" DATE,
--   "OilFcst_BBL" FLOAT,
--   "OilFcst_Start_Date" DATE
-- );
